In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap 
from mpl_toolkits.basemap import Basemap, addcyclic
from my_tools import file_dic, plot_dic
from glob import glob
from scipy.stats import ttest_ind

%matplotlib qt

In [2]:
path = '/media/onno/Volume/'
path_to_write = '/media/onno/Volume/fcst_composite/hovmoller/forecast/'

In [3]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +file_dic['envelope']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_v300_gen = xr.open_dataset(path + file_dic['v_wind']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_v300_gen =  rean_v300_gen.assign_coords(longitude=(((rean_v300_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_v300_gen=rean_v300_gen.rename({'longitude':'lon'});rean_v300_gen=rean_v300_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_v300_gen = rean_v300_gen.sel(time=[bool(i) for i in rean_v300_gen.time.dt.hour%24==0])

fcst_env_gen = xr.open_dataset(path+file_dic['envelope']['ERA5RF'],decode_times=False).squeeze()
# GFS_env_gen = xr.open_dataset(path+file_dic['envelope']['GFS'],decode_times=False).squeeze()
# ERA5RF_v300_gen = xr.open_dataset(path+'era5rf_v300_wledit2000-10000_latavg_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
# GFS_v300_gen = xr.open_dataset(path+'gefsrf2_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()

# GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
# GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
# GFS_v300_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_v300_gen.time.values]
fcst_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in fcst_env_gen.time.values]
# ERA5RF_v300_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_v300_gen.time.values]

# GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_v300_gen = GFS_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'})#; ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_v300_gen = GFS_v300_gen.assign_coords(lon=(((GFS_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
fcst_env_gen = fcst_env_gen.assign_coords(lon=(((fcst_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# ERA5RF_v300_gen = ERA5RF_v300_gen.assign_coords(lon=(((ERA5RF_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_v300_gen = GFS_v300_gen.sel(lead=GFS_v300_gen.lead.values[[bool(i) for i in GFS_v300_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# # ERA5RF_v300_gen = ERA5RF_v300_gen.sel(lead=ERA5RF_v300_gen.lead.values[[bool(i) for i in ERA5RF_v300_gen.lead%24==0]])

# GFS_env_gen['lead'] = GFS_env_gen.lead.values//24
# GFS_v300_gen['lead'] = GFS_v300_gen.lead.values//24
# ERA5RF_env_gen['lead'] = ERA5RF_env_gen.lead.values//24
# # ERA5RF_v300_gen['lead'] = ERA5RF_v300_gehttps://nos.nl/n.lead.values//24

# GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
# GFS_v300_gen = GFS_v300_gen.rename({'latitude':'lat','longitude':'lon'})
# fcst_env_gen = fcst_env_gen.rename({'latitude':'lat','longitude':'lon'})
# # ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'latitude':'lat','longitude':'lon'})





Hovmöller Diagram for RWP Envelope and V300 at any location and temperature event for best 10% of the forecasts

In [55]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}_10percent.txt'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5

    
#         datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
for model in modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        for season in seasonz:
            diffz_env_good = []
            diffz_env_bad = []
            for i,event in enumerate(eventz):
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)

                lat_max = lat_0 + 8
                lat_min = lat_1 - 10
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    rean_env_good_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                    time=slice(time_begin,time_end))
                    rean_env_good_sub = rean_env_good_sub.sel(time = rean_env_good_sub.time.values[[bool(i) for i in rean_env_good_sub.time.dt.hour%12==0]])
                    rean_env_good_sub = rean_env_good_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_good_sub.time)))
                    
                    fcst_env_good_sub = fcst_env_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=time_begin,lead=slice(0,240))
                    fcst_env_good_sub = fcst_env_good_sub.assign_coords(time=np.linspace(-lead_day,lead_day+10,len(fcst_env_good_sub.lead)))
                    
                    fcst_env_good_sub = fcst_env_good_sub.assign_coords(date=date).expand_dims('date')
                    diff_good = fcst_env_good_sub.v.squeeze() - rean_env_good_sub.v.values
                    diffz_env_good.append(diff_good)
#                 fcst_env_good = xr.concat(fcstz_env_good,dim='time').groupby('time').mean().mean('lat')
#                 fcst_v300_good = xr.concat(fcstz_v300_good,dim='time').groupby('time').mean().mean('lat')
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    rean_env_bad_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                    time=slice(time_begin,time_end))
                    rean_env_bad_sub = rean_env_bad_sub.sel(time = rean_env_bad_sub.time.values[[bool(i) for i in rean_env_bad_sub.time.dt.hour%12==0]])
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_bad_sub.time)))
                    
                    fcst_env_bad_sub = fcst_env_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=time_begin,lead=slice(0,240))
                    fcst_env_bad_sub = fcst_env_bad_sub.assign_coords(time=np.linspace(-lead_day,lead_day+10,len(fcst_env_bad_sub.lead)))
                    
                    fcst_env_bad_sub = fcst_env_bad_sub.assign_coords(date=date).expand_dims('date')
                    diff_bad = fcst_env_bad_sub.v.squeeze() - rean_env_bad_sub.v.values
                    diffz_env_bad.append(diff_bad)
#                 fcst_env_bad = xr.conccomposite_hovmoeller_all_areas_50percent_DJF.pngat(fcstz_env_bad,dim='time').groupby('time').mean().mean('lat')
#                 fcst_v300_bad = xr.concat(fcstz_v300_bad,dim='time').groupby('time').mean().mean('lat')

            diff_env_good = xr.concat(diffz_env_good,dim='date').mean('lat')
            diff_env_bad = xr.concat(diffz_env_bad,dim='date').mean('lat')
            diff_env_good.attrs['lon_0']=lon_0
            diff_env_bad.attrs['lon_0']=lon_0
            diff_env_good.to_netcdf(path_to_write+'/rawdata/diff_env_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_10percent_{}.nc'\
                                    .format(lon_0,lon_1,lat_0,lat_1,lead_day,model,season))
            diff_env_bad.to_netcdf(path_to_write+'/rawdata/diff_env_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_10percent_{}.nc'\
                                    .format(lon_0,lon_1,lat_0,lat_1,lead_day,model,season))






Combined Hovmöllers (for all regions for every type of event)

In [57]:
plt.rcParams.update({'hatch.color': 'k'})

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

modelz = ['ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
lead_day = 5
letterz_ax1 = ['a','b']
file_good = '/rawdata/diff_env_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_10percent_{}.nc'
file_bad = '/rawdata/diff_env_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_10percent_{}.nc'
seasonz = ['DJF','JJA']

for season in seasonz:
    fig,axz = plt.subplots(1,2,figsize=(16,9))
    for i,model in enumerate(modelz):
        diffz_env_good_total = []
        diffz_env_bad_total = []
        for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
            diff_env_good = xr.open_dataset(path_to_write+file_good.format(lon_0,lon_1,lat_0,lat_1,lead_day,model,season),
                                           decode_times=False)
            diff_env_good = diff_env_good.assign_coords(lon=(((diff_env_good.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_good['lead'] = diff_env_good['lead']/24 - lead_day
            diff_env_bad = xr.open_dataset(path_to_write+file_bad.format(lon_0,lon_1,lat_0,lat_1,lead_day,model,season),
                                          decode_times=False)
            diff_env_bad = diff_env_bad.assign_coords(lon=(((diff_env_bad.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_bad['lead'] = diff_env_bad['lead']/24 - lead_day
            diffz_env_good_total.append(diff_env_good)
            diffz_env_bad_total.append(diff_env_bad)
        diff_env_good_total = xr.concat(diffz_env_good_total,dim='date')
        diff_env_bad_total = xr.concat(diffz_env_bad_total,dim='date')
        diff_env_good_total.to_netcdf(path_to_write+'rawdata/diff_env_good_ALL_SEPERATE_{}_10percent_{}.nc'.format(model,season))
        diff_env_bad_total.to_netcdf(path_to_write+'rawdata/diff_env_bad_ALL_SEPERATE_{}_10percent_{}.nc'.format(model,season))
        assert(len(diff_env_good_total.date)==len(diff_env_bad_total.date))

        diff_env_good_mean = diff_env_good_total.mean('date')
        diff_env_bad_mean = diff_env_bad_total.mean('date')
        p_value_num = ttest_ind(diff_env_good_total.v.values,diff_env_bad_total.v.values,axis=0)[1]
        p_value_bool = p_value_num<0.05
        lonz = diff_env_good_mean.lon.values
        dayz = diff_env_good_mean.lead.values
        clr_levels_contourf = np.arange(-3,3,.1)
        clr_levels_contour = np.arange(-3,3,.5)
        ax1 = axz[0]
        im=ax1.contourf(lonz,dayz,diff_env_good_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
        ax1.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax1.contour(lonz,dayz,diff_env_good_mean.v,colors='k',levels=clr_levels_contour,extend='both')
        ax1.clabel(cs,inline=True,fmt='%.1f')
        ax1.set_title('Good Forecasts',fontsize=15)
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event',fontsize=12)
        ax1.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax1.set_xticks([-120,-60,0,60,120])
        ax1.set_xticklabels([])
        ax1.tick_params(labelsize=12)
        ax1.set_yticks([-5,-2.5,0,2.5,5])
        ax1.text(x=0,y=1.1,s=letterz_ax1[0]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax2 = axz[1]
        ax2.contourf(lonz,dayz,diff_env_bad_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
        ax2.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax2.contour(lonz,dayz,diff_env_bad_mean.v,colors='k',levels=clr_levels_contour,extend='both')
        ax2.clabel(cs,inline=True,fmt='%.1f')
        ax2.set_title('Bad Forecasts',fontsize=15)
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event',fontsize=12)
        ax2.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax2.set_xticks([-120,-60,0,60,120])
        ax2.set_yticks([-5,-2.5,0,2.5,5])
        ax2.tick_params(labelsize=12)
        ax2.text(x=0,y=1.05,s=letterz_ax1[1]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax1.label_outer(); ax2.label_outer()
    cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([-3,-1.5,0,1.5,3])
    cbar.ax.set_xticklabels([-3,-1.5,0,1.5,3])
    cbar.ax.set_xlabel('E (m/s)',fontsize=12)
    cbar.ax.tick_params(labelsize=12)
    fig.subplots_adjust(left=0.05,right=0.97,bottom=0.16,top=0.88,hspace=0.15,wspace=0.05)
    fig.suptitle("ERA5RF Averaged Hovmöller Good and Bad Forecasts on Day 5 Forecast {} : N = {}".format(season,len(diff_env_good_total.date)),
                fontsize=20)
    fig.savefig(path_to_write+'/figures/composite_hovmoeller_all_areas_10percent_{}.png'.format(season))
        
    

pasteCombined Hovmöllers for each region individual)

In [10]:
plt.rcParams.update({'hatch.color': 'k'})

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
lead_day = 5
letterz_ax1 = ['a','b']
letterz_ax2 = ['c','d']
file_good = '/rawdata/rean_env_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_33percent.nc'
file_bad = '/rawdata/rean_env_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_33percent.nc'


for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    fig,axz = plt.subplots(2,2,figsize=(16,9))
    for i,model in enumerate(modelz):
        rean_env_good = xr.open_dataset(path_to_write+file_good.format(lon_0,lon_1,lat_0,lat_1,lead_day,model)).load()
        rean_env_good = rean_env_good.assign_coords(lon=(((rean_env_good.lon-lon_0+180)%360)-180)).sortby('lon')
        rean_env_bad = xr.open_dataset(path_to_write+file_bad.format(lon_0,lon_1,lat_0,lat_1,lead_day,model)).load()
        rean_env_bad = rean_env_bad.assign_coords(lon=(((rean_env_bad.lon-lon_0+180)%360)-180)).sortby('lon')
        print(file_good.format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        print(file_bad.format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
#         print(float(rean_env_good.v[0,0,0]))
#         print(float(rean_env_bad.v[0,0,0]))
#         reanz_env_good_total.append(rean_env_good)
#         reanz_env_bad_total.append(rean_env_bad)
#     rean_env_good_total = xr.concat(reanz_env_good_total,dim='date')
#     rean_env_bad_total = xr.concat(reanz_env_bad_total,dim='date')
#     rean_env_good_total.to_netcdf(path_to_write+'rawdata/rean_env_good_ALL_SEPERATE_{}.nc'.format(model))
#     rean_env_bad_total.to_netcdf(path_to_write+'rawdata/rean_env_bad_ALL_SEPERATE_{}.nc'.format(model))
        assert(len(rean_env_good.date)==len(rean_env_bad.date))
        rean_env_good_mean = rean_env_good.mean('date')
        rean_env_bad_mean = rean_env_bad.mean('date')
        p_value_num = ttest_ind(rean_env_good.v.values,rean_env_bad.v.values,axis=0)[1]
        p_value_bool = p_value_num<0.1
        lonz = rean_env_good_mean.lon.values
        dayz = rean_env_good_mean.time.values
        clr_levels_contourf = np.arange(10,25.01,.1)
        clr_levels_contour = np.arange(10,25.01,1)
        ax1 = axz[0,i]
        print(float(rean_env_good_mean.v[0,0]))
        im=ax1.contourf(lonz,dayz,rean_env_good_mean.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        ax1.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax1.contour(lonz,dayz,rean_env_good_mean.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax1.clabel(cs,inline=True,fmt='%.0f')
        ax1.set_title(model_labelz[i] + '\n Good Forecasts',fontsize=15)
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event',fontsize=12)
        ax1.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax1.set_xticks([-120,-60,0,60,120])
        ax1.set_xticklabels([])
        ax1.tick_params(labelsize=12)
        ax1.set_yticks([-10,-5,0,5,10])
        ax1.text(x=0,y=1.1,s=letterz_ax1[i]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax2 = axz[1,i]
        
        ax2.contourf(lonz,dayz,rean_env_bad_mean.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        ax2.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax2.contour(lonz,dayz,rean_env_bad_mean.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax2.clabel(cs,inline=True,fmt='%.0f')
        ax2.set_title('Bad Forecasts',fontsize=15)
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event',fontsize=12)
        ax2.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax2.set_xticks([-120,-60,0,60,120])
        ax2.set_yticks([-10,-5,0,5,10])
        ax2.tick_params(labelsize=12)
        ax2.text(x=0,y=1.1,s=letterz_ax2[i]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax1.label_outer(); ax2.label_outer()
    cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([10,15,20,25])
    cbar.ax.set_xticklabels([10,15,20,25])
    cbar.ax.set_xlabel('E (m/s)',fontsize=12)
    cbar.ax.tick_params(labelsize=12)
    fig.subplots_adjust(left=0.05,right=0.97,bottom=0.16,top=0.88,hspace=0.15,wspace=0.05)
    fig.suptitle("{}-{} °N {}-{} °E Day {} Forecast: N = {}".format(lat_1,lat_0,lon_0,lon_1,lead_day,
                                                                   len(rean_env_good.date)),fontsize=20)
    sys.exit()
    fig.savefig(path_to_write+'/figures/hovmoeller_envelope_lon_{}_{}_lat_{}_{}'.format(lon_0,lon_1,lat_0,lat_1))
    plt.close(fig)
    

/rawdata/rean_env_good_lon_0_8_lat_62_54_lead_day_5_GFS_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_62_54_lead_day_5_GFS_33percent.nc
17.377910614013672
/rawdata/rean_env_good_lon_0_8_lat_62_54_lead_day_5_ERA5RF_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_62_54_lead_day_5_ERA5RF_33percent.nc
18.047258377075195


SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Hovmöller for all extreme dates

In [18]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
file = '/GFS_T850/dates/{}_lon_{}_{}_lat_{}_{}.npy'
eventz = ['pw','pc']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5

    
#         datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
for model in modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        for season in seasonz:
            for i,event in enumerate(eventz):
                diffz_env = []
                datez_file = file.format(event,lon_0,lon_1,lat_0,lat_1)
                datez = np.load(path+datez_file,allow_pickle=True)
                lat_max = lat_0 + 8
                lat_min = lat_1 - 10
                date_series = pd.to_datetime(pd.Series(datez[:,0]))
                if season == 'JJA':
                    date_series = date_series.where((date_series.dt.month>=6)&(date_series.dt.month<=8)).dropna()
                if season == 'DJF':
                    date_series = date_series.where((date_series.dt.month>=12)|(date_series.dt.month<=2)).dropna()
                print(len(date_series))
                for date in date_series:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    try:
                        rean_env_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                        time=slice(time_begin,time_end))
                        rean_env_sub = rean_env_sub.sel(time = rean_env_sub.time.values[[bool(i) for i in rean_env_sub.time.dt.hour%12==0]])
                        rean_env_sub = rean_env_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_sub.time)))

                        fcst_env_sub = fcst_env_gen.sel(lat=slice(lat_max,lat_min),
                                                       time=time_begin,lead=slice(0,240))
                        fcst_env_sub = fcst_env_sub.assign_coords(time=np.linspace(-lead_day,lead_day+10,len(fcst_env_sub.lead)))

                        fcst_env_sub = fcst_env_sub.assign_coords(date=date).expand_dims('date')
                    except KeyError:
                        continue
                    diff = fcst_env_sub.v.squeeze() - rean_env_sub.v.values
                    diffz_env.append(diff)
#                 fcst_env_good = xr.concat(fcstz_env_good,dim='time').groupby('time').mean().mean('lat')
                diff_env = xr.concat(diffz_env,dim='date').mean('lat')
                diff_env.attrs['lon_0']=lon_0
                diff_env.to_netcdf(path_to_write+'/rawdata/diff_env_lon_{}_{}_lat_{}_{}_{}_lead_day_{}_{}_{}.nc'\
                                        .format(lon_0,lon_1,lat_0,lat_1,event,lead_day,model,season))






28
23
22
32
22
22
27
24
24
27
25
28
29
24
28
31
25
25
26
28
29
31
25
31
28
22
30
35
25
26
26
30
32
27
25
35


In [4]:
plt.rcParams.update({'hatch.color': 'k'})

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

modelz = ['ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
lead_day = 5
letterz_ax1 = ['a','b']
file_pc = '/rawdata/diff_env_lon_{}_{}_lat_{}_{}_pc_lead_day_5_ERA5RF_{}.nc'
file_pw = '/rawdata/diff_env_lon_{}_{}_lat_{}_{}_pw_lead_day_5_ERA5RF_{}.nc'
seasonz = ['DJF','JJA']

for season in seasonz:
    fig,axz = plt.subplots(1,2,figsize=(16,9))
    for i,model in enumerate(modelz):
        diffz_env_pw_total = []
        len_pw = 0
        for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
            diff_env_pw = xr.open_dataset(path_to_write+file_pw.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            diff_env_pw = diff_env_pw.assign_coords(lon=(((diff_env_pw.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_pw['lead'] = diff_env_pw['lead']/24 - lead_day
            diffz_env_pw_total.append(diff_env_pw)
            print(len(diff_env_pw.date))
            len_pw += len(diff_env_pw.date)
        diff_env_pw_total = xr.concat(diffz_env_pw_total,dim='date')
        diff_env_pw_total.to_netcdf(path_to_write+'rawdata/diff_env_pw_ALL_SEPERATE_{}_{}.nc'.format(model,season))
        diff_env_pw_mean = diff_env_pw_total.mean('date')
        
        diffz_env_pc_total = []
        len_pc = 0
        for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
            diff_env_pc = xr.open_dataset(path_to_write+file_pc.format(lon_0,lon_1,lat_0,lat_1,season),
                                           decode_times=False)
            diff_env_pc = diff_env_pc.assign_coords(lon=(((diff_env_pc.lon-lon_0+180)%360)-180)).sortby('lon')
            diff_env_pc['lead'] = diff_env_pc['lead']/24 - lead_day
            diffz_env_pc_total.append(diff_env_pc)
            print(len(diff_env_pc.date))
            len_pc += len(diff_env_pc.date)
        diff_env_pc_total = xr.concat(diffz_env_pc_total,dim='date')
        diff_env_pc_total.to_netcdf(path_to_write+'rawdata/diff_env_pc_ALL_SEPERATE_{}_{}.nc'.format(model,season))
        diff_env_pc_mean = diff_env_pc_total.mean('date')
#         p_value_num = ttest_ind(diff_env_good_total.v.values,diff_env_bad_total.v.values,axis=0)[1]
#         p_value_bool = p_value_num<0.05
        lonz = diff_env_pw_mean.lon.values
        dayz = diff_env_pw_mean.lead.values
        clr_levels_contourf = np.arange(-5,5,.1)
        clr_levels_contour = np.arange(-10,10,1)
        ax1 = axz[0]
        im=ax1.contourf(lonz,dayz,diff_env_pw_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
#         ax1.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax1.contour(lonz,dayz,diff_env_diff_env_lon_0_8_lat_62_54_pw_lead_day_5_ERA5RF_JJA.ncpw_mean.v,colors='k',levels=clr_levels_contour,extend='both')
        ax1.clabel(cs,inline=True,fmt='%.1f')
        ax1.set_title('Warm Extremes : N = {}'.format(len_pw,fontsize=15))
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event',fontsize=12)
        ax1.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax1.set_xticks([-120,-60,0,60,120])
        ax1.tick_params(labelsize=12)
        ax1.set_yticks([-5,-2.5,0,2.5,5])
        ax1.text(x=0,y=1.1,s=letterz_ax1[0]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')
        ax2 = axz[1] 
        ax2.contourf(lonz,dayz,diff_env_pc_mean.v,cmap='bwr',levels=clr_levels_contourf,extend='both')
#         ax2.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax2.contour(lonz,dayz,diff_env_pc_mean.v,colors='k',levels=clr_levels_contour,extend='both')
        ax2.clabel(cs,inline=True,fmt='%.1f')
        ax2.set_title('Cold Extremes : N = {}'.format(len_pc,fontsize=15))
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event',fontsize=12)
        ax2.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax2.set_xticks([-120,-60,0,60,120])
        ax2.set_yticks([-5,-2.5,0,2.5,5])
        ax2.tick_params(labelsize=12)
        ax2.text(x=0,y=1.05,s=letterz_ax1[1]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax1.label_outer(); ax2.label_outer()
    cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([-5,-2.5,0,2.5,5])
    cbar.ax.set_xticklabels([-5,-2.5,0,2.5,5])
    cbar.ax.set_xlabel('E (m/s)',fontsize=12)
    cbar.ax.tick_params(labelsize=12)
    fig.subplots_adjust(left=0.05,right=0.97,bottom=0.16,top=0.88,hspace=0.15,wspace=0.05)
    fig.suptitle("ERA5RF Averaged Hovmöller Good and pc Forecasts on Day 5 Forecast {}",
                fontsize=20)
    sys.exit()
    fig.savefig(path_to_write+'/figures/composite_hovmoeller_all_areas_all_dates.png'.format(season))
        
    

25
26
30
25
26
28
25
27
22
35
29
34
31
27
30
28
24
31


NameError: name 'diff_env_diff_env_lon_0_8_lat_62_54_pw_lead_day_5_ERA5RF_JJA' is not defined